In [ ]:
# adapted from:
# https://ucdavis-bioinformatics-training.github.io/2018-June-RNA-Seq-Workshop/thursday/DE.html
# https://rpubs.com/jrgonzalezISGlobal/transcriptomic_analyses
# https://ucdavis-bioinformatics-training.github.io/2018-September-Bioinformatics-Prerequisites/friday/limma_biomart_vignettes.html
# https://github.com/kevinblighe/EnhancedVolcano

In [ ]:
# Dependencies

suppressWarnings(library(edgeR))
suppressWarnings(library(EnhancedVolcano))
suppressWarnings(library(patchwork)) # combine plots
suppressWarnings(library(magrittr))
suppressWarnings(library(tibble))
suppressWarnings(library(repr))
suppressWarnings(library(stringr))
suppressWarnings(library(dplyr))
suppressWarnings(library(ggplot2))
suppressWarnings(library(extrafont))
suppressWarnings(library(svglite))

suppressMessages(extrafont::font_import(pattern="Arial",prompt=FALSE))
suppressMessages(extrafont::loadfonts())

sessionInfo()

# Data import and preprocessing

In [ ]:
# Data
dir.create("figures", showWarnings = FALSE)
dir.create("out", showWarnings = FALSE)

path_to_counts =  "../data/cloud/gex/pbta-rsem-genes.expected_count.tsv"
path_to_annotation = "../data/cloud/gex/sample_phenotypes.csv"
#path_to_annotation = "out/sample_phenotypes.csv"
filter_genes <- function(gene_matrix){
    patterns <- "^ENSG|^LINC|-AS\\d$|^PAR_Y_|-DT$|-OT\\d$"
    rows_to_remove <- grep(patterns, rownames(gene_matrix))

    # Subset the matrix to exclude these rows
    filtered_data <- gene_matrix[-rows_to_remove, ]
    return (filtered_data)
}
load_inputs <- function(cts_path,annot_path){
    # Returns a list with named elements data$annot and data$cts
    message("loading data...")
    cts = as.matrix(read.csv(cts_path,sep='\t',row.names="Gene_or_Transcript_ID",check.names=FALSE))
    annot = read.csv(annot_path,row.names=1)

    cts = round(cts) # counts must be integer
    cts <- cts[, rownames(annot)] # only include samples with metadata
    new_rownames <- sub("^ENSG\\d*\\.\\d*_", "", rownames(cts)) # Remove prefixes ending with '_' from the row names
    rownames(cts) <- new_rownames # Update the row names of the matrix
    cts = filter_genes(cts) # remove common noncoding RNAs
    annot$age_at_diagnosis <- (annot$age_at_diagnosis - mean(annot$age_at_diagnosis)) / sd(annot$age_at_diagnosis)
    annot$amplified <- annot$amplicon_class %in% c('ecDNA','chromosomal')
    annot$ecDNA <- annot$amplicon_class == 'ecDNA'
    #annot <- annot[,c("sex","tumor_history","age_at_diagnosis","extent_of_tumor_resection","cancer_type","amplicon_class",'amplified','ecDNA')]

    stopifnot(all(rownames(annot) == colnames(cts)))
    return(list(cts=cts,annot=annot))
}
data=load_inputs(path_to_counts,path_to_annotation) # this will take awhile

In [ ]:
head(data$annot)

In [ ]:
head(data$cts)

In [ ]:
## lots of R pacakges work by creating a data *object* and performing matematical transformations on the structured data within.
# https://www.rdocumentation.org/packages/edgeR/versions/3.14.0/topics/DGEList-class


formula = ~ data$annot$cohort + data$annot$sex + data$annot$tumor_history + data$annot$age_at_diagnosis + data$annot$extent_of_tumor_resection + 
            data$annot$cancer_type + data$annot$amplicon_class
formula = ~ data$annot$cohort + data$annot$sex + data$annot$tumor_history + data$annot$age_at_diagnosis + data$annot$extent_of_tumor_resection + 
            data$annot$cancer_type + data$annot$amplified + data$annot$ecDNA

setup_preprocess_dge <- function(data,formula){
    design <- model.matrix(formula)
    dge <- DGEList(data$cts)
    # remove low-expressed genes
    keep <- filterByExpr(dge,design=design)
    paste("Removing",length(keep)-sum(keep),"of",length(keep),"genes with low or invariant gene expression") %>% message
    dge <- dge[keep,]
    
    # Normalize by library size
    dge <- calcNormFactors(dge,method="TMM")
    return(dge)
}

dge = setup_preprocess_dge(data,formula)

# Batch correction and outlier detection

In [ ]:
# Looks like samples cluster by cohort, with a few odd outliers.
options(repr.plot.width = 8, repr.plot.height = 6)
colors <- c("blue","red","dark green")[as.factor(data$annot$cohort)]
mds = plotMDS(dge, gene.selection = "common", col = colors, pch=16)

In [ ]:
# Corrected expression
# NB: no need to log transform before correction; identical results.
# PCs 1 and 2 driven by extreme values in a handful of outliers
dgec <- removeBatchEffect(dge,batch=as.factor(data$annot$cohort))
options(repr.plot.width = 8, repr.plot.height = 6)
colors <- c("blue","red","dark green")[as.factor(data$annot$cohort)]
mdsc = plotMDS(dgec, gene.selection = "common", col = colors, pch=16)

In [ ]:
mx = median(mdsc$x)
sx = sd(mdsc$x)
my = median(mdsc$y)
sy = sd(mdsc$y)
outlier_mask <- (mdsc$y < my-3*sy) | (mdsc$y > my+3*sy) | (mdsc$x < mx-3*sx) | (mdsc$x > mx+3*sx)
paste("Identified",sum(outlier_mask),"outlier samples")
data$annot[outlier_mask,]

In [ ]:
# Corrected expression sans outliers
# Cohorts now overlap, but PC1 describes cohort 1 variance and PC2 describes cohort 2.
options(repr.plot.width = 8, repr.plot.height = 6)
colors <- c("blue","red","dark green")[as.factor(data$annot$cohort[!outlier_mask])]
mdsc = plotMDS(dgec[,!outlier_mask], gene.selection = "common", col = colors, pch=16)

In [ ]:
data$annot[which(!(rownames(data$annot) %in% outliers)),] %>% dim

In [ ]:
# Regenerate the dataset sans outliers
# Outliers are hardcoded here if you don't want to run the previous lines of code
if (exists("outlier_mask")){
    outliers <- data$annot[outlier_mask,] %>% rownames
} else {
    outliers=c('7316-13','7316-1082','7316-2744','7316-3645','7316-170','7316-9449','7316-1089','7316-290','7316-2144','7316-1886',
               '7316-3028','7316-2589','7316-2071','7316-3657','7316-3061','7316-477','7316-2614','7316-2138','7316-291')
}
data$cts <- data$cts[,which(!(colnames(data$cts) %in% outliers))] 
data$annot <- data$annot[which(!(rownames(data$annot) %in% outliers)),]
formula = ~ data$annot$cohort + data$annot$sex + data$annot$tumor_history + data$annot$age_at_diagnosis + data$annot$extent_of_tumor_resection + 
            data$annot$cancer_type + data$annot$amplified + data$annot$ecDNA
dge = setup_preprocess_dge(data,formula)

In [ ]:
fit_dge_lm <- function(dge,design){
    # This takes a minute
    options(repr.plot.width=7, repr.plot.height=7)
    message("Fitting voom normalization...")
    v <- voom(dge, design, plot=TRUE)
    
    message("Fitting linear model...")
    fit <- lmFit(v,design)
    
    message("Calculating emperical bayes statistics...")
    fit <- eBayes(fit,robust=TRUE)
    return(fit)
}
fit <- fit_dge_lm(dge,design)

In [ ]:
# Apply multiple testing correction and obtain stats
ecDNA_comparison = ncol(design)
amp_comparison = ncol(design)-1
get_de_genes <- function(fit,comparison_index){
    print(paste("Getting DE genes w.r.t.",colnames(design)[comparison_index]))
    stats_df <- topTable(fit,n=Inf,coef=comparison_index) %>% tibble
    print(stats_df %>% head)
    return(stats_df)
}
ec_stats_df = get_de_genes(fit,ecDNA_comparison)

In [ ]:
amp_stats_df = get_de_genes(fit,amp_comparison)

In [ ]:
# Seal, R.L., Denny, P., Bruford, E.A. et al. A standardized nomenclature for mammalian histone genes. Epigenetics & Chromatin 15, 34 
# (2022). https://doi.org/10.1186/s13072-022-00467-2
# changes in v2: H1-1, H1-2, H1-3, H1-4, H1-5, H1-6, H4-16 to rdh
#                

H_pseudogenes = c('H1-9P','H1-12P',
                  'H2AC2P','H2AC3P','H2AC5P','H2AC9P','H2AC10P','H2AQ1P','H2AL1MP',
                  'H2BC2P','H2BC16P','H2BC19P','H2BC20P','H2BU2P','H2BL1P', # H2BU2P = H2BC27P
                  'H2BW3P',#'H2BW4P', # note H2BW4P and aliases not in gex dataset
                  'H3C5P','H3C9P','H3P16','H3P44',
                  'H4C10P',
                  'H2AZP7','H2AZ2P1','H2AZP2','H2AZP5','H2BP2','H2BP3','H2BP9','H4P1','H2BP1','H2AL1QP','H3P18','H3P21','H3P3','H3P1','H3P4',
                  'H3P37','H3P31','H3P14','H3P47','H3P6','H3P39','H3P13',
                  "H2AZP1","H2AZP3","H3P28","H3P27","H2ACP1","H2ACP2","H3P26" ,"H2BP7","H3P43","H2BP6","H3P45","H3P9","H3P30","H3P29","H3P10","H3P11",
                  "H3P12","H3P5","H2AZP6","H3P32","H2BP8","H3P36","H3P7","H3P2","H3P17","H3P20","H3P23","H3P22","H3P15","H3P24","H3P34","H2AZP4",
                  "H3P33","H3P35","H3P40","H3P38","H3P41","H3P8","H3P19","H3P42","H3P25","H2BP5","H3P46" ) 
H1 = c('H1-0','H1-7','H1-8','H1-10')
H1_clustered = c('H1-1','H1-2','H1-3','H1-4','H1-5','H1-6')
H2A_clustered = c('H2AC1','H2AC4','H2AC6','H2AC7','H2AC8','H2AC11','H2AC12',
                'H2AC13','H2AC14','H2AC15','H2AC16','H2AC17','H2AC18','H2AC19','H2AC20','H2AC21','H2AW') # H2AW = H2AC25
H2A = c('H2AZ1','H2AZ2','MACROH2A1','MACROH2A2','H2AX','H2AJ','H2AB1','H2AB2','H2AB3','H2AP','H2AL3')
H2B_clustered = c('H2BC1','H2BC3','H2BC4','H2BC5','H2BC6','H2BC7','H2BC8','H2BC9','H2BC10','H2BC11','H2BC12',
                 'H2BC13','H2BC14','H2BC15','H2BC17','H2BC18','H2BC21','H2BU1') # H2BU1 = H2BC26
H2B = c('H2BE1','H2BW1','H2BW2','H2BS1') # H2BS1 = H2BC12L; H2BE1 = H2BK1; H2BN1 and aliases not in gex dataset
H3_clustered = c('H3C1','H3C2','H3C3','H3C4','H3C6','H3C7','H3C8','H3C10','H3C11','H3C12','H3C13','H3C14','H3C15',
                 'H3-4') # H3-4 has a stem loop and is in a cluster
H3 = c('H3-3A','H3-3B','H3-5','H3-2','H3Y1','H3Y2','CENPA') # H3-2 = H3-7
H4_clustered = c('H4C1','H4C2','H4C3','H4C4','H4C5','H4C6','H4C7','H4C8','H4C9','H4C11','H4C12','H4C13','H4C14','H4C15',
                'H4-16') # H4-16 = H4C16 is outside the clusters but has a stem-loop and is cell-cycle regulated.
H4 = c()
rdh = c(H1_clustered,H2A_clustered,H2B_clustered,H3_clustered,H4_clustered)
nrdh = c(H1,H2A,H2B,H3,H4)
all_histones = c(rdh,nrdh,H_pseudogenes)

In [ ]:
# gene set validation checks
paste("replication-dependent histone count:",length(rdh))
paste("replication-independent histone count:",length(nrdh))
paste("histone pseudogene count:",length(H_pseudogenes))

# all genes in my sets should be in my gene expression dataset
genes <- data$cts %>% rownames
check_set <- function(name,set){
    pass = all(v %in% genes)
    print(paste(name,'...',pass))
    if (!pass){
        missing = setdiff(v,genes)
        print(paste("  missing",missing,"from gene expression data"))
    }
    return(pass)
}
vectors = list("H1ri"=H1,
          "H2Ari"=H2A,
          "H2Bri"=H2B,
          "H3ri"=H3,
          "H4ri"=H4,
          "H1rd"=H1_clustered,
          "H2Ard"=H2A_clustered,
          "H2Brd"=H2B_clustered,
          "H3rd"=H3_clustered,
          "H4rd"=H3_clustered,
          "HP"=H_pseudogenes
    )
print("Checking each histone class for genes not in expression dataset...")
flags=c()
for (name in names(vectors)){
    v = vectors[[name]]
    pass = check_set(name,v)
    if (!pass){
        flag=c(flag,name)
    }
}

print("Checking for possible unannotated histone genes in expression dataset...")
regex = "^H(1|2|3|4)|^HIST"
candidates = grep(regex,genes,value=TRUE)
candidates = setdiff(candidates,all_histones)
not_histones=c('H19')
candidates = setdiff(candidates,not_histones)
print(candidates)

In [ ]:
hits <- ec_stats_df %>% 
    #filter(adj.P.Val < 0.10) %>%
    arrange(desc(logFC))
hits %>% head(n=30)
write.table(hits, file='out/differential_expression_ecDNA.tsv',quote=FALSE,sep='\t')

In [ ]:
hits %>% tail(n=30)

In [ ]:
write_gene_sets <- function(){
    vectors = list(rdh, nrdh, H_pseudogenes)
    names = c("replication_dependent_histones","replication_independent_histones","histone_pseudogenes")
    # Open a connection to the output file
    file_conn <- file("out/histone-sets.gmt", open = "wt")
    # Write each set to the file
    for (i in seq_along(names)) {
      # Combine the name, description, and vector into a single row
      row <- c(names[i], "custom_gene_set", vectors[[i]])
      # Write the row to the file, separated by tabs
      writeLines(paste(row, collapse = "\t"), file_conn)
    }
    
    # Close the file connection
    close(file_conn)
}
write_gene_sets()

In [ ]:
## Plotting code

color_code <- list(
  "replication-dependent histones" = "blue",
  "replication-independent histones" = "darkgreen ",
  "histone pseudogenes" = "darkorange",
  "other" = "grey50",
  'c-NHEJ' = 'royalblue3',
  'Alt-EJ' = 'darkblue',
  'SSA' = 'blue3',
  'HR' = 'purple4',
  'HOXA@' = 'black',
  'HOXB@' = 'cyan',
  'HOXC@' = 'mediumorchid4',
  'HOXD@' = 'slateblue4'
)
ylabel=expression(-Log[10]*"("*italic(q)*")")

base_theme <- theme_classic(base_size=7, base_family="Arial",) +
    theme(axis.text = element_text(size=7,colour="black"))
theme_set(base_theme)

osc_volcano_ii <- function(stats_df,gene_set,gene_set_name){
    stats_df$highlight = stats_df$ID %in% gene_set
    stats_df <- stats_df[order(stats_df$highlight),]
    colCustom <- c(ifelse(stats_df$highlight, color_code[[gene_set_name]], "grey50"))
    names(colCustom) <- c(ifelse(stats_df$highlight, gene_set_name, "other"))
    plt <- EnhancedVolcano(stats_df,
                    lab = stats_df$'ID',
                    title = NULL,
                    subtitle = NULL,
                    caption = NULL,
                    axisLabSize = 14,
                    x = 'logFC',
                    y = "adj.P.Val",
                    #xlim = c(-2.75,2.75),
                    #ylim = c(0,4),
                    pCutoff = 0.05,
                    FCcutoff = 10,
                    labSize=0,
                    pointSize = c(ifelse(stats_df$highlight, 3, 1)),
                    colCustom = colCustom,
                    colAlpha = .6
                    ) %>% suppressWarnings
    options(repr.plot.width=18, repr.plot.height=7)
    return(plt + ylab(ylabel)) #+ lims(x=c(0,4),y=c(-4,4))
}

osc_volcano_v <- function(stats_df){
    highlight <- c('GFAP','HOXA6','HOXA9','HOXA10','S100B','XRCC4','CCNB1','CCNA2','CIP2A','FEN1','MYCN',
                  'RAD51','SMC2','RAD17','PRIM2','TIMELESS','CDK7','CDK1',
                  #'CENPK','AURKA','CENPN','CENPL',
                  'H3C15','H2AW','H2BU1', 'H3C13')
    stats_df$highlight = ifelse(stats_df$ID %in% highlight,"notable significant","other")
    stats_df <- stats_df[order(stats_df$highlight=='other',decreasing=TRUE),]
    stats_df$color <- ifelse(stats_df$highlight=='other','grey50','black')
    sapply(stats_df$highlight, function(x) color_code[[x]])
       
    plt <- EnhancedVolcano(stats_df,
                lab = stats_df$'ID',
                title = NULL,
                subtitle = NULL,
                caption = NULL,
                axisLabSize = 14,
                x = 'logFC',
                y = "adj.P.Val",
                #xlim = c(-2.75,2.75),
                ylim = c(0,4),
                pCutoff = 0.05,
                selectLab = highlight,
                labSize = 4.21644413212,#3.37315530569,#
                FCcutoff = 10,
                vline = NULL, 
                vlineType = "blank",
                legendPosition = "none",
                pointSize = c(ifelse(stats_df$highlight == "other", 1, 3)),
                colCustom = setNames(stats_df$color,stats_df$highlight),
                drawConnectors = TRUE,
                maxoverlapsConnectors = Inf,
                lengthConnectors = unit(0, "npc"),
                colAlpha = .6
                ) %>% suppressWarnings
    options(repr.plot.width=5, repr.plot.height=5)
    return(plt + ylab(ylabel))
}

write_plot <- function(plt,outfile,width,height){
    pdf.options(encoding='ISOLatin2.enc')
    pdfName = paste(outfile, ".pdf", sep="")
    pngName = paste(outfile, ".png", sep="")
    svgName = paste(outfile, ".svg", sep = "")
    ggsave(path="figures", filename=pdfName, device="pdf", width=width, height=height, units='in')
    ggsave(path="figures", device="png", filename=pngName, width=width, height=height, units='in')
    ggsave(path="figures", device="svg", filename=svgName, width=width, height=height, units='in')

}

In [ ]:
# Read the tab-separated file
read_gmt = function(file_path){
    data <- readLines(file_path)
    
    # Initialize an empty list to store named vectors
    named_vectors <- list()
    
    # Process each line
    for (line in data) {
      # Split the line by tab
      parts <- strsplit(line, "\t")[[1]]
      
      # The first part is the vector name
      vector_name <- parts[1]
      
      # The third part and onwards are the vector values
      vector_values <- (parts[3:length(parts)])
      
      # Create the named vector and add it to the list
      named_vectors[[vector_name]] <- vector_values
    }
    return(named_vectors)
}

In [ ]:
options(warn=-1)
plt <- osc_volcano_v(ec_stats_df)
w=14.4/4;h=3.55
options(repr.plot.width=2*w, repr.plot.height=2*h)
#write_plot(plt,"selected_volcano",w,h)
plt
options(warn=0)

In [ ]:
options(warn=-1)
p1 = osc_volcano_ii(ec_stats_df,rdh,'replication-dependent histones')
p2 = osc_volcano_ii(ec_stats_df,nrdh,'replication-independent histones')
p3 = osc_volcano_ii(ec_stats_df,H_pseudogenes, 'histone pseudogenes')
p4 = osc_volcano_ii(amp_stats_df,rdh,'replication-dependent histones')
p5 = osc_volcano_ii(amp_stats_df,nrdh,'replication-independent histones')
p6 = osc_volcano_ii(amp_stats_df,H_pseudogenes, 'histone pseudogenes')
plt <- (p1+p2+p3)/(p4+p5+p6)
w=14.4/4*3;h=9
options(repr.plot.width=w, repr.plot.height=h)
#write_plot(plt,"rdh_volcano",w,h)
plt
options(warn=0)

In [ ]:
options(warn=-1)
dsbr_sets = read_gmt("out/dsbr-sets.gmt")
plt <- osc_volcano_ii(stats_df,dsbr_sets[["c-NHEJ"]],'c-NHEJ') |
osc_volcano_ii(stats_df,dsbr_sets[["Alt-EJ"]],'Alt-EJ') |
osc_volcano_ii(stats_df,dsbr_sets[["SSA"]], 'SSA') | 
osc_volcano_ii(stats_df,dsbr_sets[["HR"]], 'HR')
w=14.4;h=4.5
options(repr.plot.width=w, repr.plot.height=h)
#write_plot(plt,"dsbr_volcano",w,h)

plt 
options(warn=0)

In [ ]:
options(warn=-1)
hox_sets = read_gmt("out/hox-sets.gmt")
plt <- osc_volcano_ii(stats_df,hox_sets[["HOXA@"]],'HOXA@') |
osc_volcano_ii(stats_df,hox_sets[["HOXB@"]],'HOXB@') |
osc_volcano_ii(stats_df,hox_sets[["HOXC@"]], 'HOXC@') | 
osc_volcano_ii(stats_df,hox_sets[["HOXD@"]], 'HOXD@')
w=14.4;h=4.5
options(repr.plot.width=w, repr.plot.height=h)
#write_plot(plt,"hox_volcano",w,h)

plt
options(warn=0)

# Old plots

In [ ]:
osc_volcano_i <- function(stats_df,gene_set,gene_set_name){
    highlight <- stats_df$ID %in% gene_set
    plt <- EnhancedVolcano(stats_df,
                    lab = stats_df$'ID',
                    title = NULL,
                    subtitle = NULL,
                    caption = NULL,
                    axisLabSize = 14,
                    x = 'logFC',
                    y = "adj.P.Val",
                    xlim = c(-3,3),
                    ylim = c(0,3),
                    pCutoff = 0.05,
                    drawConnectors = TRUE,
                    maxoverlapsConnectors = Inf,
                    #lengthConnectors = unit(2, "npc"),
                    selectLab = gene_set,
                    pointSize = c(ifelse(highlight, 3, 1)),
                    )
    options(repr.plot.width=18, repr.plot.height=7)
    return(plt)
}

osc_volcano_iii <- function(stats_df){
    stats_df$highlight = ifelse(stats_df$ID %in% rdh,"replication-dependent histones",
                                ifelse(stats_df$ID %in% nrdh, "canonical histones",
                                       ifelse(stats_df$ID %in% H_pseudogenes, "histone pseudogenes", "other")))
    stats_df <- stats_df[order(stats_df$highlight=='other',decreasing=TRUE),]
    stats_df$color <- sapply(stats_df$highlight, function(x) color_code[[x]])
                             
    plt <- EnhancedVolcano(stats_df,
                    lab = stats_df$'ID',
                    title = NULL,
                    subtitle = NULL,
                    caption = NULL,
                    axisLabSize = 14,
                    x = 'logFC',
                    y = "adj.P.Val",
                    xlim = c(-2.75,2.75),
                    ylim = c(0,3),
                    pCutoff = 0.05,
                    FCcutoff = 10,
                    labSize=0,
                    pointSize = c(ifelse(stats_df$highlight == "other", 1, 3)),
                    colCustom = setNames(stats_df$color,stats_df$highlight)
                    )
    options(repr.plot.width=8, repr.plot.height=9)
    return(plt + ylab(ylabel)) #+ lims(x=c(0,4),y=c(-4,4))
}

osc_volcano_iv <- function(stats_df){
    plt <- EnhancedVolcano(stats_df,
                lab = stats_df$'ID',
                title = NULL,
                subtitle = NULL,
                caption = NULL,
                axisLabSize = 14,
                x = 'logFC',
                y = "adj.P.Val",
                xlim = c(-2.75,2.75),
                ylim = c(0,3),
                pCutoff = 0.05,
                drawConnectors = TRUE,
                maxoverlapsConnectors = Inf,
                lengthConnectors = unit(0, "npc"),   
                )
    options(repr.plot.width=12, repr.plot.height=12)
    return(plt)
}

In [ ]:
osc_volcano_i(ec_stats_df,rdh) +
osc_volcano_i(ec_stats_df,nrdh) + 
osc_volcano_i(ec_stats_df,H_pseudogenes)

In [ ]:
plt <- osc_volcano_iv(ec_stats_df)
#write_plot(plt,"sig_volcano",11,11)
plt